In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from constants import BASE_SAVE_DIR, SAVE_EXTENSION, PROJECT_NAME_BASE, FONTSIZE
from utils import render_in_latex, set_fontsize, get_save_path, get_runs, get_metrics_and_colors, get_metric_statistics, plot_metric_statistics, bar_metric_statistics

In [3]:
dataset = "houseelec"
metric = "test_posterior_samples_mean_nll"
x_axis_name = "time"

In [4]:
set_fontsize(FONTSIZE)
render_in_latex()

In [5]:
runs = get_runs(PROJECT_NAME_BASE + dataset)
metrics_dict, colors_dict = get_metrics_and_colors(runs, metric)
statistics_dict = get_metric_statistics(metrics_dict)
save_path_plot = get_save_path(f"{BASE_SAVE_DIR}/{dataset}/{metric}", f"{x_axis_name}.{SAVE_EXTENSION}")
save_path_bar = get_save_path(
    f"{BASE_SAVE_DIR}/{dataset}/{metric}", f"bar.{SAVE_EXTENSION}")

/home/pratikr/scalable_gp_inference/plotting/utils.py:127: UserWarning: Not all MetricData objects have the same steps. This may lead to incorrect results. This is likely because some runs were not finished. We will return None for the mean, min, and max data.
  warnings.warn(
/home/pratikr/scalable_gp_inference/plotting/utils.py:201: UserWarning: Skipping SDD-100 due to inconsistent metric data across runs.
  warnings.warn(


In [6]:
plot_metric_statistics(statistics_dict, colors_dict, x_axis_name, dataset, save_path=save_path_plot)

In [ ]:
bar_metric_statistics(statistics_dict, colors_dict, dataset, save_path=save_path_bar)